# CSV to Parquet Conversion

Parquet has a much smaller footprint allowing you to reduce storage space and improve performance when loading the data to memory. This transformation is particularly important for out-of-memory computation with increased IO for large datasets. Parquet is column-based storage, making column-based operations particularly effective. 

## Initialization

In [1]:
import dask.dataframe as dd
import numpy as np
import dask.delayed
import pyarrow

In [2]:
from pathlib import PurePath

input_directory = "../data/"
filename = '2018_Yellow_Taxi_Trip_Data'
extension = '.csv'
csv_sep = ','
input_file = PurePath(input_directory, filename + extension)

output_directory = PurePath(input_directory, filename)
output_filename_base = filename

## Start local Dask client

In [3]:
from dask.distributed import Client, LocalCluster
try:
    if client:
        print('Restarting client')
        client.restart()
except:
#     cluster = LocalCluster(dashboard_address=':20100', memory_limit='4G')
    cluster = LocalCluster(dashboard_address=':20100')
    print('Setting new client')
    client = Client(cluster)
    print(client)
client

Setting new client
<Client: 'tcp://127.0.0.1:36259' processes=5 threads=10, memory=25.61 GB>


Client Scheduler: tcp://127.0.0.1:36259 Dashboard: http://127.0.0.1:20100/status,Cluster Workers: 5 Cores: 10 Memory: 25.61 GB


## Get all available columns

In [4]:
ddf = dd.read_csv(input_file, sep=csv_sep)
columns = ddf.columns.values
for i, column in enumerate(columns):
    print(str(i) + ': ' + column)

0: VendorID
1: tpep_pickup_datetime
2: tpep_dropoff_datetime
3: passenger_count
4: trip_distance
5: RatecodeID
6: store_and_fwd_flag
7: PULocationID
8: DOLocationID
9: payment_type
10: fare_amount
11: extra
12: mta_tax
13: tip_amount
14: tolls_amount
15: improvement_surcharge
16: total_amount


## Data Interface

In [10]:
categorical_features = ['VendorID', 
                        'RatecodeID', 
                        'PULocationID',
                        'DOLocationID',
                        'payment_type',
#                         'store_and_fwd_flag'
                       ]
datetime_features = ['tpep_pickup_datetime',
                    'tpep_dropoff_datetime']

numerical_features = ['passenger_count', 
                     'trip_distance', 
                     'fare_amount', 
                      'extra',
                      'mta_tax',
                      'tip_amount',
                      'tolls_amount',
                      'improvement_surcharge',
                      'total_amount'
                     ]

# Type dict to improve dynamic loading of csv
dtypes = {**{col: 'category' for col in categorical_features}, \
         **{col: 'float64' for col in numerical_features}}

In [11]:
columns_to_load = categorical_features + datetime_features + numerical_features 

In [12]:
ddf = dd.read_csv(input_file, 
                  usecols = columns_to_load, 
                  dtype=dtypes, 
                  sep=csv_sep, 
                  parse_dates = datetime_features)
#                 , 
#                   blocksize = 32e6)

## Preprocessing

In [14]:
"""
Convert to int32 in cents in order to not lose precision 
"""
# ddf[numerical_features] = ddf[numerical_features] * 100

In [16]:
# ddf.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-12-03 09:58:01,2018-12-03 10:14:17,100.0,120.0,1,186,161,1,1100.0,0.0,50.0,295.0,0.0,30.0,1475.0
1,2,2018-12-03 09:41:32,2018-12-03 10:20:08,100.0,1203.0,1,138,162,1,3900.0,0.0,50.0,911.0,576.0,30.0,5467.0
2,2,2018-12-03 08:54:36,2018-12-03 08:59:35,200.0,86.0,1,151,166,1,550.0,0.0,50.0,126.0,0.0,30.0,756.0
3,2,2018-12-03 09:02:08,2018-12-03 09:07:16,200.0,109.0,1,166,238,1,600.0,0.0,50.0,136.0,0.0,30.0,816.0
4,2,2018-12-03 09:10:10,2018-12-03 09:21:32,200.0,178.0,1,238,75,1,950.0,0.0,50.0,206.0,0.0,30.0,1236.0


In [21]:
# Cast to int now that this is in a cent representation
# ddf[numerical_features] = ddf[numerical_features].astype(dtype='int32')

In [22]:
# ddf.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-12-03 09:58:01,2018-12-03 10:14:17,100,120,1,186,161,1,1100,0,50,295,0,30,1475
1,2,2018-12-03 09:41:32,2018-12-03 10:20:08,100,1203,1,138,162,1,3900,0,50,911,576,30,5467
2,2,2018-12-03 08:54:36,2018-12-03 08:59:35,200,86,1,151,166,1,550,0,50,126,0,30,756
3,2,2018-12-03 09:02:08,2018-12-03 09:07:16,200,109,1,166,238,1,600,0,50,136,0,30,816
4,2,2018-12-03 09:10:10,2018-12-03 09:21:32,200,178,1,238,75,1,950,0,50,206,0,30,1236


In [26]:
ddf.dtypes

VendorID                       category
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int32
trip_distance                     int32
RatecodeID                     category
PULocationID                   category
DOLocationID                   category
payment_type                   category
fare_amount                       int32
extra                             int32
mta_tax                           int32
tip_amount                        int32
tolls_amount                      int32
improvement_surcharge             int32
total_amount                      int32
dtype: object

## Write parquet file in modified format

In [27]:
def create_parquet_file(ddf, output_filepath):
    dd.to_parquet(ddf, output_filepath)

In [28]:
%time create_parquet_file(ddf, output_directory)

KeyboardInterrupt: 

## Test performance in Dask file reading for both file types

In [21]:
import fastparquet
import dask.dataframe as ddf
# output_directory = PurePath('datetime_yellow_taxi_parquet')
test_ddf = dd.read_parquet(output_directory)
# print(test_ddf['tpep_pickup_datetime'].head())
# output_directory

## Parquet vs CSV reading speed test

In [10]:
csv_ddf = dd.read_csv(input_file, usecols = columns_to_load, dtype=dtypes, sep=csv_sep)
pq_ddf = dd.read_parquet(output_directory, usecols = columns_to_load, dtype=dtypes)

In [11]:
def average_result(ddf):
    # Have to transform to float64 to prevent overflow and inf outcomes
    ddf['trip_distance'] = ddf['trip_distance'].astype(np.float64)
    return ddf.groupby('VendorID')['trip_distance'].mean()

### CSV 

In [12]:
# File size is ~ 10Gb
%time result = average_result(csv_ddf).compute()    
print(result)

CPU times: user 5.59 s, sys: 565 ms, total: 6.16 s
Wall time: 1min 26s
VendorID
1    2.791033
2    3.031729
4    2.703563
Name: trip_distance, dtype: float64


### Parquet

In [13]:
# Parquet directory size ~ 2.9G
%time result = average_result(pq_ddf).compute()    
print(result)

KeyboardInterrupt: 

VendorID
1    2.791033
2    3.031729
4    2.703563
Name: trip_distance, dtype: float64


In [14]:
### Parquet
import dask.dataframe as dd
ddf_reduced_float = dd.read_parquet('../data/2018_Yellow_Taxi_Trip_Data')
ddf_full_float = dd.read_parquet('../data/2018_Yellow_Taxi_Trip_Data_float64')

In [15]:
ddf_reduced_float.dtypes

VendorID                       category
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                     category
store_and_fwd_flag             category
PULocationID                   category
DOLocationID                   category
payment_type                   category
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
dtype: object

In [16]:
ddf_full_float.dtypes

VendorID                       category
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                     category
store_and_fwd_flag               object
PULocationID                   category
DOLocationID                   category
payment_type                   category
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
dtype: object

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Interstingly, the reduced float size seems to increase file size (more than 2x). Possibly not worth doing. Lets check memory footprint

In [ ]:
df_reduced_float = ddf_reduced_float.get_partition(0)
df_reduced_float.info(memory_usage='deep')

In [ ]:
df_full_float = ddf_full_float.get_partition(0)
df_full_float.info(memory_usage='deep')

In [ ]:
%time result = average_result(ddf_full_float).compute()    
print(result)